#### SQL запросы для создания и редактирования таблиц в БД

In [ ]:
CREATE TABLE current_orders (
    token TEXT NOT NULL,
    exchange TEXT NOT NULL,
    market_type TEXT NOT NULL,
    order_type TEXT NOT NULL,
    order_side TEXT NOT NULL,
	qty NUMERIC(18,8) NOT NULL,
    price NUMERIC(18,8) NOT NULL,
    usdt_amount NUMERIC(18,10),
    usdt_fee NUMERIC(18,8) NOT NULL,
	leverage NUMERIC(4,1) NOT NULL,
    created_at TIMESTAMPTZ DEFAULT now(),
    PRIMARY KEY (token, exchange, market_type)
);

CREATE TABLE trading_history (
    id SERIAL PRIMARY KEY,
    token TEXT NOT NULL,
    exchange TEXT NOT NULL,
    market_type TEXT NOT NULL,
    order_type TEXT NOT NULL,
    order_side TEXT NOT NULL,
    open_price NUMERIC(18,8) NOT NULL,
    close_price NUMERIC(18,8) NOT NULL,
    open_usdt_amount NUMERIC(18,10),
    close_usdt_amount NUMERIC(18,10),
    qty NUMERIC(18,8) NOT NULL,
    open_fee NUMERIC(18,8) NOT NULL,
    close_fee NUMERIC(18,8) NOT NULL,
    leverage NUMERIC(4,1) NOT NULL,
    created_at TIMESTAMPTZ,
    closed_at TIMESTAMPTZ DEFAULT now(),
    profit NUMERIC(18,8)
);

In [ ]:
# 
# SELECT * FROM market_data_5s
# WHERE 
#     exchange = 'bybit' 
#     AND market_type = 'linear' 
#     AND token = 'ADA_USDT'
# ORDER BY bucket DESC;

In [ ]:
# SELECT remove_retention_policy('market_data');

SELECT add_retention_policy(
    'market_data', 
    drop_after => INTERVAL '5 minutes',
    schedule_interval => INTERVAL '5 minutes'
);

In [ ]:
-- Список всех таблиц с их размерами
SELECT 
    relname AS "Таблица",
    pg_size_pretty(pg_total_relation_size(relid)) AS "Полный размер"
FROM pg_catalog.pg_statio_user_tables
ORDER BY pg_total_relation_size(relid) DESC;

In [ ]:
CREATE MATERIALIZED VIEW market_data_5s
WITH (timescaledb.continuous) AS
SELECT 
    time_bucket('5 seconds', timestamp) AS bucket,
    exchange,
    market_type,
    token,
    AVG(bid_price) AS avg_bid,
    AVG(ask_price) AS avg_ask
FROM market_data
GROUP BY bucket, exchange, market_type, token;

In [ ]:
SELECT add_continuous_aggregate_policy(
  'market_data_5s',
  start_offset => INTERVAL '1 hour',
  end_offset => INTERVAL '5 seconds',
  schedule_interval => INTERVAL '10 seconds'
);

#### PostgreSQL

In [ ]:
%load_ext autoreload
%autoreload 2

from datetime import datetime, timedelta
from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.db.redis_manager import RedisManager

import pandas as pd
import polars as pl
from jaref_bot.config.credentials import host, user, password, db_name

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)
redis_order_manager = RedisManager(db_name = 'orders')

db_manager.get_auto_copy_trigger_state()

In [ ]:
# db_manager.set_auto_copy_trigger_state('enable')

In [ ]:
# db_manager.clear_table('market_data_5s')

In [ ]:
db_manager.get_candlestick(exchange='bybit', token='ETH_USDT', return_type='polars')

In [ ]:
datetime.now() - datetime(2025, 8, 23, 18, 0)

In [ ]:
# db_manager.clear_old_data('orderbook_1h', column='bucket', expiration_time=240, units='hours')

In [ ]:
# db_manager.get_public_trades(exchange='bybit', token='ETH_USDT', return_type='polars')

#### Orderbooks

In [ ]:
ob = db_manager.get_orderbooks(exchange='bybit', market_type='linear', symbol='AKT_USDT', interval='1min')

In [ ]:
ob

In [ ]:
db_manager.get_table('orderbook_1min')

In [ ]:
db_manager.get_table('raw_orderbook_data').tail(10)

In [ ]:
# db_manager.clear_table('orderbook_1min')

#### Orders & trading_history

In [ ]:
db_manager.clear_table('trading_history')
db_manager.clear_table('current_orders')
redis_order_manager.clear_orders_table()

In [ ]:
db_manager.delete_order('ADA_USDT', 'okx', 'linear')

In [ ]:
db_manager.place_order(token='ADA_USDT', exchange='okx', market_type='linear', order_type='market', order_side='buy',
                  qty=160, price=0.5927, usdt_amount=94.672, realized_pnl=-0.0025, leverage=1)

In [ ]:
# db_manager.close_order(token='SUI_USDT', exchange='bybit', market_type='linear', qty=30.0,
#                        close_price=2.98, close_usdt_amount=59.6, close_fee=0.021456)

In [ ]:
current_orders = db_manager.get_table('current_orders')
current_orders

In [ ]:
trading_history = db_manager.get_table('trading_history')
trading_history

In [ ]:
trading_history['profit'].sum()

#### Redis

In [ ]:
%load_ext autoreload
%autoreload 2

import redis
from redis.exceptions import ConnectionError
import polars as pl
import pandas as pd
from datetime import datetime
from decimal import Decimal
import orjson
import json

redis_client = redis.Redis(db=0, decode_responses=True)

from jaref_bot.db.redis_manager import RedisManager

redis_manager = RedisManager(db_name='orderbooks')

try: 
    redis_client.ping()
    print('Сервер Redis запущен')
except ConnectionError:
    print('Сервер Redis не отвечает')

In [ ]:
redis_client.flushdb()

In [ ]:
redis_client.keys()

In [ ]:
ts = redis_client.hget("orderbook:bybit:linear:ZBCN_USDT:update_time", 'cts')
datetime.fromtimestamp(ts)

In [ ]:
redis_client.hgetall("orderbook:bybit:linear:ZBCN_USDT")

In [ ]:
dd = redis_manager.get_orderbooks(1)
dd